In [1]:
#Add folder to sys path to import modules from sub folders
import sys
sys.path.insert(0, r'C:\Users\SaANTIAGO\Google Drive Streaming\My Drive\18_UT_RAPID\cuttings_sensor')

In [2]:
from src.text_extraction import *
#from src.file_dialog import *
# import numpy as np
# import cv2
# from tkinter import Tk, filedialog
# from pathlib import Path
# import os
# from os import listdir
# from os.path import isfile, join
# from PIL import Image, ImageEnhance, ImageOps, ImageFilter
# import PIL.Image
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = Path(r'C:\Program Files\Tesseract-OCR\tesseract.exe') #Set Pytesseract path
# #import keras_ocr
# import pandas as pd
# import datetime

import matplotlib.pyplot as plt

# Image Processing References

https://pythongeeks.org/image-processing-in-python/

https://www.section.io/engineering-education/image-adjustment-to-higher-or-lower-resolution-using-python/

https://www.kaggle.com/competitions/shopee-product-matching/discussion/225463

In [3]:
#Get Directory and Files Paths
directory, files = get_directory_files(file_extension='.tiff')

In [4]:
#Get number and frame number lists
number_list = get_number_list(files)
frame_number_list = get_frame_list(files)

KeyboardInterrupt: 

In [ ]:
#Set Dataframe
video_data = pd.DataFrame(np.array(number_list), columns=['laser_measurement'])
frame_number_df = pd.DataFrame(np.array(frame_number_list), columns=['frame_number'])

In [ ]:
#Get Arrays
time_array = get_time_index(initial_time_str='2022-06-22 16:20:04', seconds=1, files=files)
conveyor_array = get_conveyor_index(initial_conveyor=73600, distance=160, files=files)
#Get DFs
time_array_df = pd.DataFrame(time_array, columns=['time_index'])
conveyor_df = pd.DataFrame(conveyor_array, columns=['conveyor_index'])

In [ ]:
#Join Dataframes
video_data = video_data.join(frame_number_df)
video_data = video_data.join(time_array_df)
video_data = video_data.join(conveyor_df)

In [ ]:
#Save Dataframe
save_dataframe('video_2_2022_06_22_data.csv', video_data)